In [1]:
import numpy as np
import pandas as pd
import torch
import sys

sys.path.append("../")

## Loading Model and Vocabulary

In [2]:
folder = "weights/cbow_own4"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = torch.load(f"../{folder}/model.pt", map_location=device)
vocab = torch.load(f"../{folder}/vocab.pt")

## Getting Embeddings

In [3]:
# embedding from first model layer
embeddings = list(model.parameters())[0]
embeddings = embeddings.cpu().detach().numpy()

# normalization
norms = (embeddings ** 2).sum(axis=1) ** (1 / 2)
norms = np.reshape(norms, (len(norms), 1))
embeddings_norm = embeddings / norms
embeddings_norm.shape

# Find Similar Words

In [5]:
def get_top_similar(word: str, topN: int = 10):
    word_id = vocab[word]
    if word_id == 0:
        print("Out of vocabulary word")
        return

    word_vec = embeddings_norm[word_id]
    word_vec = np.reshape(word_vec, (len(word_vec), 1))
    dists = np.matmul(embeddings_norm, word_vec).flatten()
    topN_ids = np.argsort(-dists)[1 : topN + 1]

    topN_dict = {}
    for sim_word_id in topN_ids:
        sim_word = vocab.lookup_token(sim_word_id)
        topN_dict[sim_word] = dists[sim_word_id]
    return topN_dict

In [20]:
for word, sim in get_top_similar("paper").items():
    print("{}: {:.3f}".format(word, sim))

article: 0.925
dissertation: 0.789
chapter: 0.666
poster: 0.653
thesis: 0.647
letter: 0.645
manuscript: 0.631
demo: 0.588
monograph: 0.579
work: 0.518


# Vector Equations

In [23]:
emb1 = embeddings[vocab["math"]]
emb2 = embeddings[vocab["mathematics"]]
emb3 = embeddings[vocab["biology"]]

emb4 = emb1 - emb2 + emb3
emb4_norm = (emb4 ** 2).sum() ** (1 / 2)
emb4 = emb4 / emb4_norm

emb4 = np.reshape(emb4, (len(emb4), 1))
dists = np.matmul(embeddings_norm, emb4).flatten()

top5 = np.argsort(-dists)[:5]

for word_id in top5:
    print("{}: {:.3f}".format(vocab.lookup_token(word_id), dists[word_id]))

biology: 0.620
math: 0.546
genomics: 0.367
biologists: 0.358
multigpu: 0.346
